# Offset

In [1]:
import threading
import time
import numpy as np
import sounddevice as sd
import serial
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import matplotlib.pyplot as plt
from ordered_set import OrderedSet
import pickle

pygame 2.5.2 (SDL 2.28.3, Python 3.11.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [5]:
ESP_COM_PORT = "COM23"
ESP_BAUDRATE = 115200
TIME_PERIOD = 0.125
ref_db = 0.51

In [3]:
# Get the default audio playback device
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)

# Create a volume control interface
volume_controller = cast(interface, POINTER(IAudioEndpointVolume))

In [15]:
# Function to receive serial data and store it
def receive_serial_data(data):
    ser = serial.Serial(ESP_COM_PORT, ESP_BAUDRATE)  # Replace 'COM1' with your serial port
    while not terminate_event.is_set():
        received_data = ser.readline().decode().strip()
        received_data = received_data.split(" ")[0]
        try:
            received_data = float(received_data)
        except ValueError:
            received_data = float(received_data.split(".")[0])
        try:
            if log_flag:
                data = np.append(data, received_data)
        except KeyError:
            pass
    ser.close()

In [ ]:
# Function to play audio sine wave
def play_test_wave():
    duration = 15 if TIME_PERIOD == 0.125 else 30 # s
    sample_rate = 88200
    t = np.arange(0, duration, 1 / sample_rate)

    sine_wave = np.sin(2 * np.pi * 1000 * t)
    volume_controller.SetMasterVolumeLevelScalar(ref_db, None)

    log_flag = True

    # Wait before playing the sine wave
    time.sleep(0.375 if TIME_PERIOD == 0.125 else 3) #s

    sd.play(sine_wave, sample_rate, blocking=True)

    # Wait after playback is done
    time.sleep(0.375 if TIME_PERIOD == 0.125 else 3) #s

    log_flag = False

    # Send a signal to the other thread to terminate
    terminate_event.set()

In [7]:
# Create formatted container to store received serial data
offset_data = np.array([])

# Event to signal termination
terminate_event = threading.Event()
# Flag to signal when to log measurements
log_flag = False

## Ejecutar prueba

In [ ]:
# Create the serial data receiving thread
serial_thread = threading.Thread(target=receive_serial_data, args=(offset_data,))
serial_thread.daemon = True

# Create the audio playback thread
audio_thread = threading.Thread(target=play_test_wave)
audio_thread.daemon = True

# Start serial thread
serial_thread.start()
# Play tests
audio_thread.start()

# Wait for the threads to finish
serial_thread.join()
audio_thread.join()

# Reset the event for next test
terminate_event.clear()

## Guardar datos

## Procesar datos del sonómetro

In [ ]:
# Convert the SLM reference data to the same format as the ESP measurements
reference = {}

## Analizar datos

In [42]:
# TODO: Delete
# Test datasets
test1 = np.random.uniform(33, 92, size=(100,))
test2 = np.random.uniform(33, 92, size=(100,))

In [50]:
# Plot the sound level progression with respect to time, for each frequency
def plot_time_progression(measurements, reference, units, title):
    for freq in measurements.frequencies:
        # Create x-axis values
        x_axis = [x * TIME_PERIOD for x in range(len(measurements))]

        # Create a plot
        plt.plot(x_axis, measurements, label="ESP")
        plt.plot(x_axis, reference, label="SLM")

        # Add labels and a title (optional)
        plt.xlabel("s")
        plt.ylabel(units)
        plt.title(f"Sound level time progression" + title)
        plt.legend()

        # Show the plot
        plt.show()

In [ ]:
def calculate_offset(measurements, reference):
    if len(measurements) != len(reference):
        raise ValueError("Both sets must have the same length")
    offset = np.mean(measurements - reference)
    return offset